**5P02 - Peer Review**

**Hannah Thomas**

In [ ]:
from psychopy import visual, event, core, data, gui
from psychopy.tools.filetools import fromFile
import random
import numpy as np

# Experiment setup
expName = 'VisualSearch'
dlg = gui.Dlg()
dlg.addField('SubjectID:')
dlg.addField('Trials Per Cond:')
ok_data = dlg.show()
if not dlg.OK:
    core.quit()


For this experiment setup, I am using an older version of Psychopy (v2024.1.5) but when I tried to utilize this feature it did not work for me, which might be due to my version of Psychopy (see forum here that explains the functionality): https://www.psychopy.org/api/gui.html

In order to have the label show (i.e., SubjectID:), I needed to add dlg.addText above dlg.addField in order to have a label appear. 

Once I figured out what values I was entering, I had to modify the indexing used below to finally get the experiment to run:

In [ ]:
sub_ID = ok_data[0]
trials = int(dlg.data[1])
fileName = sub_ID + "_" + expName
dataFile = open(fileName + '.csv', 'w')
dataFile.write('SetSize,TP, RT, Correct, Missed\n') # NOTE: For analyses, you should store the subject ID into the data file as well. 

Again, I could not get this to work (another reason I think it might be a version issue). Here, you are indexing sub_ID by position by using ok_data[0] which is the first value inputted into the dlg list. For it to work for me, I needed to index by the name of the value like sub_ID = ok_data['SubjectID:']. I'm not entirely sure why this didn't work for me, but it was a helpful practice in debugging someone elses code. 

I've run into instances before where I've downloaded experiments/analyses scripts from other researchers, only for it not to work for me. It was a good learning experience to try and get the experiment to run for me, especially now that I have more knowledge about what could potentially be causing the issue (i.e., indexing).

In [ ]:
win = visual.Window([1920, 1080], fullscr=False, units='pix') 

# Stimuli and conditions
conditions = [5, 8, 12]
stim_size = 30
T = visual.ImageStim(win, 'Stimuli/T.png', size=stim_size)
L = visual.ImageStim(win, 'Stimuli/L.png', size=stim_size)

I did not use pixels for my units, but I had to specify the height and width of my stimuli. It seems by using stim_size = 30, it automatically makes the height = 30 and width = 30. Because you are using pixel units, your stimuli should not change size depending on your screen size. I spent a lot of time trying to fix the aspect ratio of my stimuli, but this appears to be a more roboust method. However, I believe that by using pixels the quality of the image might vary based on the monitor? 

In [ ]:
# Draw stimuli at random positions/orientations
def pos_and_ori(target, distract, samp_size):
    samplelist = list(range(-180, 180, 25))
    x = random.sample(samplelist, samp_size)
    y = random.sample(samplelist, samp_size)
    for n in range(0, samp_size - 1):
        orientations = [0, 90, 180, 270]
        orin = random.choice(orientations)
        distract.ori = orin
        distract.pos = (x[n], y[n])
        distract.draw()
    for n in range(samp_size - 1, samp_size):
        target.pos = (x[n], y[n])
        target.draw()
    return distract, target

I'm impressed that this was placed into a function. 

The code does work, but I noticed that you draw the distract and target here, when they are not yet specified as visual stimuli. I believe you can just have the function assign the pos and orientation and then return those values. 

One thing I didn't understand is where the code is retrieving the samp_size value from, I understand that is one of the arguments it takes but by changing it to condition later on it, it makes it appear that these are different things. Using one name consistently would improve readability (i.e., set_size). 

In [ ]:
# Determine if target is present on a given trial
def targ_pres(trial_list, total_trials, distract, target, condition_index):
    pres_or_not = random.choice(trial_list)
    trial_list.remove(pres_or_not)
    if pres_or_not <= np.median(total_trials):
        targ_there = 0
        stimuli = pos_and_ori(distract, distract, condition)
    else:
        targ_there = 1
        stimuli = pos_and_ori(target, distract, condition)
    return targ_there, stimuli

This was something I struggled to figure out, how to have the target appear on 50% of trials. The use of np.median(total_trials): makes a lot of sense. It will account for trial blocks with odd numbers. 

The logic appears to be: if the chosen trial number is less than or equal to the median of all trials then the trial will be a target absent.
Else, if the chosen trial number is greater than the median (I guess the opposite of the if) then the trial will be target present. 

Based on that logic, the target argument from pos_and_ori is replaced with distract. I think this makes sense why you drew the stimulus earlier (but it still hasn't been assigned as a visual stimulus). 

Additionally, the target_press takes the argument condition_index, which is a another name for the same thing (i.e., set_size). 

In [ ]:
# Get response and RT
def KeyGet(trial_duration=2.0, rt=None, resp=None):
    startTime = core.getTime()
    while core.getTime() - startTime < trial_duration and resp is None:
        keys = event.getKeys(keyList=['a', 'd', 'escape'])
        if keys:
            key = keys[0]
            rt = core.getTime() - startTime
            if key == 'a':
                resp = 'a'
                break
            elif key == 'd':
                resp = 'd'
                break
            elif key == 'escape':
                core.quit()
        core.wait(0.01)
    if resp is None:
        resp = 'no_response'
        rt = 999
    return resp, rt

My only critique here is that usually on visual search tasks (based on my knowledge) you would have participants withhold their response on target absent trials. And it's easier on participants to have the key correspond to the letter as well (i.e., use T instead). Response key bindings are important!

I will say I do not know enough about the timing functions to critique whether core.getTime() - startTime is the best method for getting reaction time, but I think just rt = respClock.getTime() from lecture 5 would suffice? And making sure that you properly reset the clock after each trial. 

In [ ]:
# Evaluate response accuracy and feedback
def Response(resp, rt, targ_there):
    if resp == 'd' and targ_there == 1:
        corr = 1
        feedback = 'Correct!'
        response_time = round(rt, 2)
    elif resp == 'a' and targ_there == 1:
        corr = 0
        feedback = 'Incorrect!'
        response_time = round(rt, 2)
    elif resp == 'a' and targ_there == 0:
        corr = 1
        feedback = 'Correct!'
        response_time = round(rt, 2)
    elif resp == 'd' and targ_there == 0:
        corr = 0
        feedback = 'Incorrect'
        response_time = round(rt, 2)
    elif resp == 'no_response':
        corr = 0
        feedback = 'No Response'
        response_time = 'NA'
    return corr, feedback, response_time

Good use of function to include this info, I like that you were able to print the RT as well. 

Smart to use round to display the rt to 2 decimal places, very detail oriented. 

In [ ]:
# Instructions
welcome = ''''
Welcome to the Visual Search Task

You will see an assortment of shapes in different positions and orientations.
Most will be 'L' shapes, but some may contain a 'T' shape.

If the T is present, press 'd'.
If the T is absent, press 'a'.

Respond quickly!
Press SPACE to begin 5 practice trials.
'''

instructions = visual.TextStim(win, color='white', text=welcome, units='norm', height=0.05)
instructions.draw()
win.flip()
keys = event.waitKeys(keyList=['space'])
core.wait(0.25)

Good use of the ''' to display multiple lines of text, I actually entered in a page break using \.n and this seems much easier!

One thing here is that the scale units have been switched to norm, I would suggest using consistent units throughout your experiment. 

In [ ]:
# Practice trials
practice_trials = range(1, 6)
for condition in conditions:
    appear = list(practice_trials)
    for prac_trials in practice_trials:
        targ_there, stimuli = targ_pres(appear, practice_trials, L, T, condition)
        resp, rt = KeyGet()
        #win.flip() <- adding this here will display the trial, then the feedback on the next trial. 
        corr, feedback, response_time = Response(resp, rt, targ_there)
        cor_feedback = visual.TextStim(win, text=feedback, pos=(0, 30), height=40)
        back_rt = visual.TextStim(win, text=response_time, pos=(0, -30), height=40)
        cor_feedback.draw()
        back_rt.draw()
        win.flip()
        core.wait(0.5)

In the practice loop and main experiment loop, a critical point here is that the trial itself displayed at the same time as the feedback. This is also a limitation of drawing the stimuli earlier. I have included where you should add the win.flip for this to work properly.

Additionally, the number of practice trials was not 5, but instead the 5 * the # of conditions = 3 since you loop through conditions. 

In [ ]:
# Main experiment instructions
welcome = '''' 
Practice complete! Now for the real trials.

Press SPACE to begin.
'''
instructions = visual.TextStim(win, color='white', text=welcome, units='pix', height=10)
instructions.draw()
win.flip()
keys = event.waitKeys(keyList=['space'])
core.wait(0.25)

Instead of creating the instructions stimulus again, you could do instructions.text=welcomeMain to update the text to your new main experiment instructions. 

In [ ]:
# Trial setup
total_trials = range(1, trials + 1)
rt_list, corr_list, miss_rt_list = [], [], []
random.shuffle(conditions)

You create the total_trials variable near the bottom of your script, but it is used earlier when creating the function targ_pres. If all it is doing is specifying your number of trials taken from the gui.dlg then I believe you can create it earlier when you create the trials variable. I think it works currently bc you set total_trials as an argument but do not utilize it until before you create the variable. 

I tried to explore ways to improve this but your trials are not completely random, they are set up in "blocks" of set sizes that are randomized, but it is not truly random. 

In [ ]:
# Main experiment loop
for condition in conditions:
    appear = list(total_trials)
    for trial in total_trials:
        targ_there, stimuli = targ_pres(appear, total_trials, L, T, condition)
        resp, rt = KeyGet()
        # win.flip() <- needs to be added here as well
        corr, feedback, response_time = Response(resp, rt, targ_there)
        cor_feedback = visual.TextStim(win, text=feedback, pos=(0, 30), height=40)
        back_rt = visual.TextStim(win, text=response_time, pos=(0, -30), height=40)
        cor_feedback.draw()
        back_rt.draw()
        win.flip()
        core.wait(0.5)

        if rt != 999:
            rt_list.append(rt)
            miss_rt = 0
            corr_list.append(corr)
        else:
            miss_rt_list.append(1)
            miss_rt = 1
            corr_list.append(0)

        dataFile.write('%i, %i, %.3f, %i, %i\n' % (condition, targ_there, rt, corr, miss_rt))

dataFile.close()

Smart to only include your main experimental trials in your data file. I think it would be benefical to store the feedback and keys pressed to determine exactly what kind of error was made. We typically differentiate between hits (correct responses), misses (missing the signal) and false alarms (incorrectly pressing 'd' when you should press 'a'). 

I couldn't figure out what stimuli was being used for in this loop, it seems unnecessary?

In [ ]:
# Final feedback
average_rt = round(np.mean(rt_list), 2)
average_corr = round(np.mean(corr_list), 2)
total_miss = sum(miss_rt_list)

avg_rt_text = f'average rt: {average_rt}'
avg_corr_text = f'average correct: {average_corr}'
miss_text = f'no response on {total_miss} trials'
leave_text = 'press SPACE to exit'

cor_avg_back = visual.TextStim(win, text=avg_corr_text, pos=(0, 50), height=20)
rt_avg_back = visual.TextStim(win, text=avg_rt_text, pos=(0, -10), height=20)
miss_tot_back = visual.TextStim(win, text=miss_text, pos=(0, -35), height=20)
exit_text = visual.TextStim(win, text=leave_text, pos=(0, -100), height=20)

cor_avg_back.draw()
rt_avg_back.draw()
miss_tot_back.draw()
exit_text.draw()

win.flip()
keys = event.waitKeys(keyList=['space'])
win.close()
core.quit()


I was really impressed with this, since I could not figure it out for my assignment. I see by creating the rt_list, corr_list, and miss_rt_list within the loop and then appending the responses to them you were able to include overall feedback once the experimental loop had finished, awesome stuff.

**Overall Feedback**

Overall, this students assignment did an excellent job tackling the challenges of the assignment. I was impressed by the use of functions and also how they did not use trialHandler or experimentHandler demonstrating good knowledge of how to make trials without built-in functions, cool! 

One reoccuring critique is the name of some variables/arguments were not consistent, and some did not appear to be used. For example, sample_size, condition, and condition_index were used but I think they all referred to the set size. 

Some inconsistencies in scale units throughout the code.

There appeared to be a lot of variables used in earlier functions that were specified later on, which because they were technically unsued they did not create errors but in general I think it's important to always create variables in a logical order to avoid using a function, value, etc. that the program hasn't encountered yet. 

I learned a lot from this assignment, and I think by reviewing this work and trying to debug it I've gain a better understanding of python. 

For future courses: I think this assignment would be an excellent practice to include more than once, since I think debugging code is half of the battle and can aid in comprehension. 